In [ ]:
using ProgressMeter, PyCall, PyPlot, Cairo, Images, Optim, HDF5, Statistics, Lasso, CoordinateTransformations, OffsetArrays, Printf
using _Data, _Math
using Statistics, NaNStatistics, Distributions, StatsBase, HypothesisTests, GLM, DataFrames, DSP, Interpolations, LsqFit, FixedPointNumbers, Clustering

In [ ]:
@pyimport numpy
@pyimport scipy.stats as stats
@pyimport pandas as pd
@pyimport matplotlib.font_manager as font_manager

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
# include("../../../functions/func_map.jl")
# include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/utils.jl")

## Test on one fish

In [ ]:
experimenter = "jen"
analyzer = "chuyu"
server = 9

experiment_filename = "20220406_111526"
ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save = Dataset(experiment_filename, analyzer, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

C, heading, img_bg, y_fish, x_offset, x_fish, y_offset = h5open(ds, "behavior.h5"; raw = true) do file
    read(file, "C"),
    read(file, "heading"), 
    read(file, "img_bg"),
    read(file, "fish_anchor_y"), 
    read(file, "offset_x"), 
    read(file, "fish_anchor_x"), 
    read(file, "offset_y")
end;

w = size(img_bg, 1)
l = size(img_bg, 2);

In [ ]:
#roi averged activity, which wasalready saved
neuron_merge_activity_file = h5open(joinpath(data_path(ds_save), "NMF_merge.h5"))
A_dF = HDF5.readmmap(neuron_merge_activity_file["A_dF"]);
close(neuron_merge_activity_file)
n_neurons = size(A_dF, 2);

In [ ]:
NMF_filename = joinpath(data_path(ds_save), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
close(NMF_file)
n_neuron = n_neurons

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end

In [ ]:
tel_neurons = findall(whether_tel);

In [ ]:
n_sweeps = size(A_dF, 1)

In [ ]:
x_fish_sweep = reshape(x_fish[1:n_sweeps*125], (125, n_sweeps,))
y_fish_sweep = reshape(y_fish[1:n_sweeps*125], (125, n_sweeps,));

x_fish_sweep_mean = dropdims(mean(x_fish_sweep, dims = 1), dims = 1)
y_fish_sweep_mean = dropdims(mean(y_fish_sweep, dims = 1), dims = 1);

figure; scatter(x_fish_sweep_mean, y_fish_sweep_mean,c=1:length(x_fish_sweep_mean))

In [ ]:
x_fish_mm_fit = fit(TrendFilter, x_fish_sweep_mean .* 0.02, 5, 2);
y_fish_mm_fit = fit(TrendFilter, y_fish_sweep_mean .* 0.02, 5, 2);

figure(figsize = (16,3)); 
plot(x_fish_sweep_mean .* 0.02, linewidth = 0.5)

plot(x_fish_mm_fit.β, linewidth = 0.5)
ylim(0, 100)
xlim(0, 1000)

In [ ]:
speed_mm_s = (diff(x_fish_mm_fit.β).^2 .+ diff(y_fish_mm_fit.β).^2).^0.5 ./ 0.5;

speed_mm_s = vcat(speed_mm_s[1], speed_mm_s);
figure(figsize = (16,3)); plot(speed_mm_s, linewidth = 0.5)
# xlim(0, 1000)

In [ ]:
speed_thr = 0.1
valid_moving_indices = (speed_mm_s .> speed_thr);
plot(valid_moving_indices)

In [ ]:
heading_sweep = reshape(heading[1:n_sweeps*125], (125, n_sweeps,))
heading_sweep_mean = dropdims(mean(heading_sweep, dims = 1), dims = 1);
figure(figsize = (16,3)); plot(heading_sweep_mean, linewidth = 0.5)

In [ ]:
n_pos = 15 #here you can specify the number of bins, bins are equally size in x and y, indicating spatial resolution

min_x = floor(Int64, minimum(x_fish_sweep_mean));
max_x = floor(Int64, maximum(x_fish_sweep_mean));

min_y = floor(Int64, minimum(y_fish_sweep_mean));
max_y = floor(Int64, maximum(y_fish_sweep_mean));

interval = maximum([(max_y-min_y+2)/n_pos,(max_x-min_x+2)/n_pos])

# x_bins = LinRange(min_x, max_x, n_pos+1)
# y_bins = LinRange(min_y, max_y, n_pos+1);
x_bins = collect(min_x-1:interval:min_x+interval*(n_pos)+1);
y_bins = collect(min_y-1:interval:min_y+interval*(n_pos)+1);

x_bins_mid = (x_bins[1:end-1]+x_bins[2:end])/2
y_bins_mid = (y_bins[1:end-1]+y_bins[2:end])/2;


# Digitize data, then we just count the number
x_digital = numpy.digitize(x_fish_sweep_mean, x_bins)
y_digital = numpy.digitize(y_fish_sweep_mean, y_bins);
loc_digital = (y_digital.-1).*n_pos.+x_digital;

loc_vectors = zeros(Int32, maximum(loc_digital),n_sweeps)
for i in 1:maximum(loc_digital)
    loc_vectors[i,findall(loc_digital.==i)] .= 1
end

In [ ]:
n_angle = 24

angle_bins = numpy.linspace(0, 2*pi, n_angle+1);

heading_digital = numpy.digitize(heading_sweep_mean, angle_bins)

heading_vectors = zeros(Int32, n_angle,n_sweeps)
for i in 1:n_angle
    heading_vectors[i,findall(heading_digital.==i)] .= 1
end

In [ ]:
n_speed = 24

speed_bins = numpy.linspace(0, maximum(speed_mm_s), n_speed+1);

speed_digital = numpy.digitize(speed_mm_s, speed_bins)

speed_vectors = zeros(Int32, n_speed,n_sweeps)
for i in 1:n_speed
    speed_vectors[i,findall(speed_digital.==i)] .= 1
end

In [ ]:
# Valid indices for calculation
bool_index = trues(n_sweeps)
bool_index[.!valid_moving_indices].=false;

In [ ]:
heading_glm_paramters = fill(NaN32,1+size(heading_vectors,1), n_neurons)
heading_r_2_all = fill(NaN32,n_neurons)
@showprogress for neuron_index in tel_neurons
    neural_activity = A_dF[:,neuron_index]
    valid_time_this_neuron = .!(isnan.(neural_activity));
    valid_this_neuron = findall(valid_time_this_neuron.*bool_index);
    mod = sm.OLS((A_dF[valid_this_neuron,neuron_index]),(sm.add_constant(heading_vectors'))[valid_this_neuron,:])
    mod_results = mod.fit_regularized(L1_wt=0.0, alpha=1e-10)
    heading_glm_paramters[:,neuron_index] .= mod_results.params
    predicted_activity = mod_results.predict();
    heading_r_2_all[neuron_index] = sklearn.metrics.r2_score(neural_activity[valid_this_neuron],predicted_activity)
end

In [ ]:
speed_glm_paramters = fill(NaN32,1+size(speed_vectors,1), n_neurons)
speed_r_2_all = fill(NaN32,n_neurons)
@showprogress for neuron_index in tel_neurons
    neural_activity = A_dF[:,neuron_index]
    valid_time_this_neuron = .!(isnan.(neural_activity));
    valid_this_neuron = findall(valid_time_this_neuron.*bool_index);
    mod = sm.OLS((A_dF[valid_this_neuron,neuron_index]),(sm.add_constant(speed_vectors'))[valid_this_neuron,:])
    mod_results = mod.fit_regularized(L1_wt=0.0, alpha=1e-10)
    speed_glm_paramters[:,neuron_index] .= mod_results.params
    predicted_activity = mod_results.predict();
    speed_r_2_all[neuron_index] = sklearn.metrics.r2_score(neural_activity[valid_this_neuron],predicted_activity)
end

In [ ]:
loc_glm_paramters = fill(NaN32,1+size(loc_vectors,1), n_neurons)
loc_r_2_all = fill(NaN32,n_neurons)
@showprogress for neuron_index in tel_neurons
    neural_activity = A_dF[:,neuron_index]
    valid_time_this_neuron = .!(isnan.(neural_activity));
    valid_this_neuron = findall(valid_time_this_neuron.*bool_index);
    mod = sm.OLS((A_dF[valid_this_neuron,neuron_index]),(sm.add_constant(loc_vectors'))[valid_this_neuron,:])
    mod_results = mod.fit_regularized(L1_wt=0.0, alpha=1e-10)
    loc_glm_paramters[:,neuron_index] .= mod_results.params
    predicted_activity = mod_results.predict();
    loc_r_2_all[neuron_index] = sklearn.metrics.r2_score(neural_activity[valid_this_neuron],predicted_activity)
end

In [ ]:
tog_vectors = vcat(heading_vectors, loc_vectors, speed_vectors);

In [ ]:
tog_glm_paramters = fill(NaN32,1+size(tog_vectors,1), n_neurons)
tog_r_2_all = fill(NaN32,n_neurons)
@showprogress for neuron_index in tel_neurons
    neural_activity = A_dF[:,neuron_index]
    valid_time_this_neuron = .!(isnan.(neural_activity));
    valid_this_neuron = findall(valid_time_this_neuron.*bool_index);
    mod = sm.OLS((A_dF[valid_this_neuron,neuron_index]),(sm.add_constant(tog_vectors'))[valid_this_neuron,:])
    mod_results = mod.fit_regularized(L1_wt=0.0, alpha=1e-10)
    tog_glm_paramters[:,neuron_index] .= mod_results.params
    predicted_activity = mod_results.predict();
    tog_r_2_all[neuron_index] = sklearn.metrics.r2_score(neural_activity[valid_this_neuron],predicted_activity)
end

In [ ]:
h5open(joinpath(data_path(ds_save), "ols_results_tel.h5"), "w") do file
    file["heading_glm_paramters"] = heading_glm_paramters
    file["heading_r_2_all"] = heading_r_2_all
    file["speed_glm_paramters"] = speed_glm_paramters
    file["speed_r_2_all"] = speed_r_2_all
    file["loc_glm_paramters"] = loc_glm_paramters
    file["loc_r_2_all"] = loc_r_2_all
    file["tog_glm_paramters"] = tog_glm_paramters
    file["tog_r_2_all"] = tog_r_2_all
end;

In [ ]:
hist(loc_r_2_all, alpha=0.3, bins=100, density=true,label="loc");
hist(speed_r_2_all, alpha=0.3, bins=30, density=true,label="speed");
hist(heading_r_2_all, alpha=0.3, bins=100, density=true,label="heading");
hist(tog_r_2_all, alpha=0.3, bins=100, density=true,label="heading+loc+speed");

xlabel("r2")
legend()
xlim(-0.01, 1)
title("OLS")
xlim(0,0.4)

In [ ]:
file_folder_1 = joinpath(data_path(ds_save), "")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = [occursin("A_dF", long_name_files[i])*occursin("neuron", long_name_files[i]) for i in 1:length(long_name_files)]
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_cell_index = HDF5.readmmap(file["place_cell_index"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
close(file)

In [ ]:
hist(loc_r_2_all[place_cell_index], alpha=0.3, bins=100, density=true,label="loc");
hist(speed_r_2_all[place_cell_index], alpha=0.3, bins=30, density=true,label="speed");
hist(heading_r_2_all[place_cell_index], alpha=0.3, bins=100, density=true,label="heading");
hist(tog_r_2_all[place_cell_index], alpha=0.3, bins=100, density=true,label="heading+loc+speed");

xlabel("r2")
legend()
xlim(-0.01, 1)
title("OLS")
# xlim(0,0.4)

In [ ]:
hist(loc_r_2_all, alpha=0.3, bins=100, density=true,label="loc");
hist(speed_r_2_all, alpha=0.3, bins=30, density=true,label="speed");
hist(heading_r_2_all, alpha=0.3, bins=100, density=true,label="heading");
hist(tog_r_2_all, alpha=0.3, bins=100, density=true,label="heading+loc+speed");

xlabel("r2")
legend()
xlim(-0.01, 1)
title("OLS")
xlim(0,0.4)

In [ ]:
NMF_filename = joinpath(data_path(ds_save), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
close(NMF_file)

n_neuron = length(X_all);
valid_neurons  = 1:n_neuron;

In [ ]:
plot_quantity(heading_r_2_all./tog_r_2_all;label="heading/tog  $experiment_filename")

In [ ]:
plot_quantity(speed_r_2_all./tog_r_2_all;label="speed/tog  $experiment_filename")

In [ ]:
plot_quantity(loc_r_2_all;vmax=0.25,vmin=0,label="r2, loc")

In [ ]:
plot_quantity(loc_r_2_all ./ tog_r_2_all;label="loc/tog  $experiment_filename")

# For all fish

In [ ]:
data_info_all = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 4, "jen"],
    ["20220416_160516", 6, "jen"]
];

In [ ]:
analyzer = "chuyu"

In [ ]:
for data_info in data_info_all
experimenter = data_info[3]
server = data_info[2]
experiment_filename = data_info[1]
    
ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save = Dataset(experiment_filename, analyzer, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

C, heading, img_bg, y_fish, x_offset, x_fish, y_offset = h5open(ds, "behavior.h5"; raw = true) do file
    read(file, "C"),
    read(file, "heading"), 
    read(file, "img_bg"),
    read(file, "fish_anchor_y"), 
    read(file, "offset_x"), 
    read(file, "fish_anchor_x"), 
    read(file, "offset_y")
end;

w = size(img_bg, 1)
l = size(img_bg, 2);

#roi averged activity, which wasalready saved
NMF_filename = joinpath(data_path(ds_save), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
A_dF = HDF5.readmmap(NMF_file["A_dF"]);
Z_all = HDF5.readmmap(NMF_file["Z_all"])
X_all = HDF5.readmmap(NMF_file["X_all"])
Y_all = HDF5.readmmap(NMF_file["Y_all"])
close(NMF_file)
n_neurons = size(A_dF, 2);
n_neuron = size(A_dF, 2);

NMF_filename = joinpath(data_path(ds_save), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
close(NMF_file)

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end

tel_neurons = findall(whether_tel);


n_sweeps = size(A_dF, 1)


x_fish_sweep = reshape(x_fish[1:n_sweeps*125], (125, n_sweeps,))
y_fish_sweep = reshape(y_fish[1:n_sweeps*125], (125, n_sweeps,));

x_fish_sweep_mean = dropdims(mean(x_fish_sweep, dims = 1), dims = 1)
y_fish_sweep_mean = dropdims(mean(y_fish_sweep, dims = 1), dims = 1);


x_fish_mm_fit = fit(TrendFilter, x_fish_sweep_mean .* 0.02, 5, 2);
y_fish_mm_fit = fit(TrendFilter, y_fish_sweep_mean .* 0.02, 5, 2);

speed_mm_s = (diff(x_fish_mm_fit.β).^2 .+ diff(y_fish_mm_fit.β).^2).^0.5 ./ 0.5;

speed_mm_s = vcat(speed_mm_s[1], speed_mm_s);


speed_thr = 0.1
valid_moving_indices = (speed_mm_s .> speed_thr);

heading_sweep = reshape(heading[1:n_sweeps*125], (125, n_sweeps,))
heading_sweep_mean = dropdims(mean(heading_sweep, dims = 1), dims = 1);


n_pos = 15 #here you can specify the number of bins, bins are equally size in x and y, indicating spatial resolution

min_x = floor(Int64, minimum(x_fish_sweep_mean));
max_x = floor(Int64, maximum(x_fish_sweep_mean));

min_y = floor(Int64, minimum(y_fish_sweep_mean));
max_y = floor(Int64, maximum(y_fish_sweep_mean));

interval = maximum([(max_y-min_y+2)/n_pos,(max_x-min_x+2)/n_pos])

# x_bins = LinRange(min_x, max_x, n_pos+1)
# y_bins = LinRange(min_y, max_y, n_pos+1);
x_bins = collect(min_x-1:interval:min_x+interval*(n_pos)+1);
y_bins = collect(min_y-1:interval:min_y+interval*(n_pos)+1);

x_bins_mid = (x_bins[1:end-1]+x_bins[2:end])/2
y_bins_mid = (y_bins[1:end-1]+y_bins[2:end])/2;


# Digitize data, then we just count the number
x_digital = numpy.digitize(x_fish_sweep_mean, x_bins)
y_digital = numpy.digitize(y_fish_sweep_mean, y_bins);
loc_digital = (y_digital.-1).*n_pos.+x_digital;

loc_vectors = zeros(Int32, maximum(loc_digital),n_sweeps)
for i in 1:maximum(loc_digital)
    loc_vectors[i,findall(loc_digital.==i)] .= 1
end


n_angle = 24

angle_bins = numpy.linspace(0, 2*pi, n_angle+1);

heading_digital = numpy.digitize(heading_sweep_mean, angle_bins)

heading_vectors = zeros(Int32, n_angle,n_sweeps)
for i in 1:n_angle
    heading_vectors[i,findall(heading_digital.==i)] .= 1
end


n_speed = 24

speed_bins = numpy.linspace(0, maximum(speed_mm_s), n_speed+1);

speed_digital = numpy.digitize(speed_mm_s, speed_bins)

speed_vectors = zeros(Int32, n_speed,n_sweeps)
for i in 1:n_speed
    speed_vectors[i,findall(speed_digital.==i)] .= 1
end


# Valid indices for calculation
bool_index = trues(n_sweeps)
bool_index[.!valid_moving_indices].=false;


heading_glm_paramters = fill(NaN32,1+size(heading_vectors,1), n_neurons)
heading_r_2_all = fill(NaN32,n_neurons)
@showprogress for neuron_index in tel_neurons
    neural_activity = A_dF[:,neuron_index]
    valid_time_this_neuron = .!(isnan.(neural_activity));
    valid_this_neuron = findall(valid_time_this_neuron.*bool_index);
    mod = sm.OLS((A_dF[valid_this_neuron,neuron_index]),(sm.add_constant(heading_vectors'))[valid_this_neuron,:])
    mod_results = mod.fit_regularized(L1_wt=0.0, alpha=1e-10)
    heading_glm_paramters[:,neuron_index] .= mod_results.params
    predicted_activity = mod_results.predict();
    heading_r_2_all[neuron_index] = sklearn.metrics.r2_score(neural_activity[valid_this_neuron],predicted_activity)
end

speed_glm_paramters = fill(NaN32,1+size(speed_vectors,1), n_neurons)
speed_r_2_all = fill(NaN32,n_neurons)
@showprogress for neuron_index in tel_neurons
    neural_activity = A_dF[:,neuron_index]
    valid_time_this_neuron = .!(isnan.(neural_activity));
    valid_this_neuron = findall(valid_time_this_neuron.*bool_index);
    mod = sm.OLS((A_dF[valid_this_neuron,neuron_index]),(sm.add_constant(speed_vectors'))[valid_this_neuron,:])
    mod_results = mod.fit_regularized(L1_wt=0.0, alpha=1e-10)
    speed_glm_paramters[:,neuron_index] .= mod_results.params
    predicted_activity = mod_results.predict();
    speed_r_2_all[neuron_index] = sklearn.metrics.r2_score(neural_activity[valid_this_neuron],predicted_activity)
end


loc_glm_paramters = fill(NaN32,1+size(loc_vectors,1), n_neurons)
loc_r_2_all = fill(NaN32,n_neurons)
@showprogress for neuron_index in tel_neurons
    neural_activity = A_dF[:,neuron_index]
    valid_time_this_neuron = .!(isnan.(neural_activity));
    valid_this_neuron = findall(valid_time_this_neuron.*bool_index);
    mod = sm.OLS((A_dF[valid_this_neuron,neuron_index]),(sm.add_constant(loc_vectors'))[valid_this_neuron,:])
    mod_results = mod.fit_regularized(L1_wt=0.0, alpha=1e-10)
    loc_glm_paramters[:,neuron_index] .= mod_results.params
    predicted_activity = mod_results.predict();
    loc_r_2_all[neuron_index] = sklearn.metrics.r2_score(neural_activity[valid_this_neuron],predicted_activity)
end

tog_vectors = vcat(heading_vectors, loc_vectors, speed_vectors);


tog_glm_paramters = fill(NaN32,1+size(tog_vectors,1), n_neurons)
tog_r_2_all = fill(NaN32,n_neurons)
@showprogress for neuron_index in tel_neurons
    neural_activity = A_dF[:,neuron_index]
    valid_time_this_neuron = .!(isnan.(neural_activity));
    valid_this_neuron = findall(valid_time_this_neuron.*bool_index);
    mod = sm.OLS((A_dF[valid_this_neuron,neuron_index]),(sm.add_constant(tog_vectors'))[valid_this_neuron,:])
    mod_results = mod.fit_regularized(L1_wt=0.0, alpha=1e-10)
    tog_glm_paramters[:,neuron_index] .= mod_results.params
    predicted_activity = mod_results.predict();
    tog_r_2_all[neuron_index] = sklearn.metrics.r2_score(neural_activity[valid_this_neuron],predicted_activity)
end


h5open(joinpath(data_path(ds_save), "ols_results_tel.h5"), "w") do file
    file["heading_glm_paramters"] = heading_glm_paramters
    file["heading_r_2_all"] = heading_r_2_all
    file["speed_glm_paramters"] = speed_glm_paramters
    file["speed_r_2_all"] = speed_r_2_all
    file["loc_glm_paramters"] = loc_glm_paramters
    file["loc_r_2_all"] = loc_r_2_all
    file["tog_glm_paramters"] = tog_glm_paramters
    file["tog_r_2_all"] = tog_r_2_all
    file["tel_neurons"] = tel_neurons

end;


end

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure1/ols"

In [ ]:
loc_r_2_tog = []
heading_r_2_tog = []
speed_r_2_tog = []
tog_r_2_tog = []
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experimenter = data_info[3]
    experiment_filename = data_info[1]
    server = data_info[2]
    ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
    ds_save_cy = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")


    h5open_all(h5open(joinpath(data_path(ds_save_cy), "ols_results_tel.h5"), "r"))
    
    
    file_folder_1 = joinpath(data_path(ds_save_cy), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)




    place_cell_index = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01))

    
    
    append!(speed_r_2_tog, speed_r_2_all[place_cell_index])
    append!(loc_r_2_tog, loc_r_2_all[place_cell_index])
    append!(heading_r_2_tog, heading_r_2_all[place_cell_index])
    append!(tog_r_2_tog, tog_r_2_all[place_cell_index])

end

In [ ]:
ols_result = Dict()
ols_result["Heading"] = heading_r_2_tog
ols_result["Position"] = loc_r_2_tog
ols_result["Speed"] = speed_r_2_tog
ols_result["All"] = tog_r_2_tog;
df = pd.DataFrame(ols_result)

In [ ]:
@pyimport matplotlib.cm as cm 
order = ["Speed", "Heading", "Position", "All"]

bins_raw= numpy.linspace(0,1,100)

tab10 = cm.get_cmap("tab10", 10)

colors = [tab10(i) for i in [3,0,1,2]]

fig, ax = subplots(1,1,figsize=(1.5,0.8), dpi=250)
for (i, key_) in enumerate(order)
    first_bin = findfirst(bins_raw.>nanminimum(Float32.(ols_result[key_])))
    last_bin = findfirst(bins_raw.>nanmaximum(Float32.(ols_result[key_])))
    println(bins_raw[first_bin])
    println(nanminimum(Float32.(ols_result[key_])))
    println(bins_raw[last_bin])
    println(nanmaximum(Float32.(ols_result[key_])))
    bins = bins_raw[first_bin: last_bin]
    global count_1, _ = hist(ols_result[key_], label=key_,bins=bins, histtype="stepfilled", fc=(colors[i][1], colors[i][2], colors[i][3], 0.3),ec=colors[i], linewidth=0.7)
    println(maximum(count_1))
end
ax.legend(frameon=false,loc=(0.6, 0.1))
xlabel("\$R^2\$")
ylabel("Count",labelpad=-12)
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
xlim(0,1)
ylim(-100,1500)
# yscale("log")
# yticks([1,10,100,1000],["\$10^0\$","","","\$10^3\$"])
yticks([0, 500, 1000, 1500],["0","","","1500"])
title("Tel. place cells", fontsize=7)

fig.savefig(joinpath(fig_path, "ols_regression_tel.pdf"), bbox_inches = "tight",pad_inches = 0, transparent = true)

In [ ]:
loc_r_2_tog = []
heading_r_2_tog = []
speed_r_2_tog = []
tog_r_2_tog = []
for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experimenter = data_info[3]
    experiment_filename = data_info[1]
    server = data_info[2]
    ds = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
    ds_save_cy = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")


    h5open_all(h5open(joinpath(data_path(ds_save_cy), "ols_results_tel.h5"), "r"))
    
    
    file_folder_1 = joinpath(data_path(ds_save_cy), "")
    all_files = readdir(file_folder_1)
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(file_folder_1, save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)




    place_cell_index = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01))
    
    
    other_cell_index = tel_neurons[.!whether_in(tel_neurons, place_cell_index)]
    
    
    
    append!(speed_r_2_tog, speed_r_2_all[other_cell_index])
    append!(loc_r_2_tog, loc_r_2_all[other_cell_index])
    append!(heading_r_2_tog, heading_r_2_all[other_cell_index])
    append!(tog_r_2_tog, tog_r_2_all[other_cell_index])

end

In [ ]:
ols_result = Dict()
ols_result["Heading"] = heading_r_2_tog
ols_result["Position"] = loc_r_2_tog
ols_result["Speed"] = speed_r_2_tog
ols_result["All"] = tog_r_2_tog;
df = pd.DataFrame(ols_result)

In [ ]:
@pyimport matplotlib.cm as cm 
order = ["Speed", "Heading", "Position", "All"]

bins_raw= numpy.linspace(0,1,100)

tab10 = cm.get_cmap("tab10", 10)

colors = [tab10(i) for i in [3,0,1,2]]

fig, ax = subplots(1,1,figsize=(1.5,0.8), dpi=250)
for (i, key_) in enumerate(order)
    first_bin = findfirst(bins_raw.>nanminimum(Float32.(ols_result[key_])))
    last_bin = findfirst(bins_raw.>nanmaximum(Float32.(ols_result[key_])))
    println(bins_raw[first_bin])
    println(nanminimum(Float32.(ols_result[key_])))
    println(bins_raw[last_bin])
    println(nanmaximum(Float32.(ols_result[key_])))
    bins = bins_raw[first_bin: last_bin]
    global count_1, _ = hist(ols_result[key_], label=key_,bins=bins, histtype="stepfilled", fc=(colors[i][1], colors[i][2], colors[i][3], 0.3),ec=colors[i], linewidth=0.7)
    println(maximum(count_1))
end
ax.legend(frameon=false,loc=(0.6, 0.1))
xlabel("\$R^2\$")
ylabel("Count",labelpad=-12)
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
xlim(0,1)
ylim(-500,11000)
# yscale("log")
# yticks([1,10,100,1000],["\$10^0\$","","","\$10^3\$"])
yticks([0, 10000],["0","\$10^4\$"])
title("Tel. other cells", fontsize=7)
fig.savefig(joinpath(fig_path, "ols_regression_tel_other.pdf"), bbox_inches = "tight",pad_inches = 0, transparent = true)